# Advanced RNN - 3
- ```CuDNNGRU``` & ```CuDNNLSTM``` implementation will make training much faster
- **Note:** in TF2.0, you don't need to use ```CuDNNGRU``` and ```CuDNNLSTM``` - the default GRU and LSTM will run on GPU if conditions are met

In [ ]:
from keras.datasets import imdb
from keras.layers import GRU, LSTM, CuDNNGRU, CuDNNLSTM, Activation
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

### Import dataset
- IMDB dataset in Keras datasets
- doc: https://keras.io/datasets/

In [ ]:
num_words = 30000
maxlen = 300

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = num_words)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000,)
(25000,)
(25000,)
(25000,)


In [ ]:
# pad the sequences with zeros 
# padding parameter is set to 'post' => 0's are appended to end of sequences
X_train = pad_sequences(X_train, maxlen = maxlen, padding = 'post')
X_test = pad_sequences(X_test, maxlen = maxlen, padding = 'post')

In [ ]:
X_train = X_train.reshape(X_train.shape + (1,))
X_test = X_test.reshape(X_test.shape + (1,))

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(25000, 300, 1)
(25000, 300, 1)
(25000,)
(25000,)


### LSTM
- Naive LSTM model without CuDNN

In [ ]:
def lstm_model():
    model = Sequential()
    model.add(LSTM(50, input_shape = (300,1), return_sequences = True))
    model.add(LSTM(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
model = lstm_model()

In [ ]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

Wall time: 29min 40s


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 50.84%


### GRU
- Naive GRU model without CuDNN

In [ ]:
def gru_model():
    model = Sequential()
    model.add(GRU(50, input_shape = (300,1), return_sequences = True))
    model.add(GRU(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
model = gru_model()

In [ ]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

Wall time: 21min 46s


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 52.40%


### CuDNN LSTM

In [ ]:
def cudnn_lstm_model():
    model = Sequential()
    model.add(CuDNNLSTM(50, input_shape = (300,1), return_sequences = True))
    model.add(CuDNNLSTM(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
model = cudnn_lstm_model()

In [ ]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

Wall time: 2min 53s


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 51.70%


### CuDNN GRU

In [ ]:
def cudnn_gru_model():
    model = Sequential()
    model.add(CuDNNGRU(50, input_shape = (300,1), return_sequences = True))
    model.add(CuDNNGRU(1, return_sequences = False))
    model.add(Activation('sigmoid'))
    
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
model = cudnn_gru_model()

In [ ]:
%%time
model.fit(X_train, y_train, batch_size = 100, epochs = 10, verbose = 0)

Wall time: 1min 54s


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 51.60%
